### Urban Circularity

Script Version 1

This script does not contain the archetype assignment logic.
The logic needs to be updated and will be revised in the following(newer) versions of the pipeline.

This script is written based on information available on the AllData dataset for city of Ithaca.
It successfully organizes the large output dataframe into smaller subsets for easier accessibility. 

##### 1. Importing libraries

In [1]:
import warnings
warnings.filterwarnings("ignore")

# Libraries to work with datasets
import pandas as pd
import numpy as np
import geopandas as gpd
import fiona
import h5py

import os

# Libraries to plot/edit maps 
import matplotlib.pyplot as plt

##### 2. Fetching and Cleaning data

##### Method 1: Identifying roof types

This definition identifies whether the building may have flat roofs or pitched roofs. 

In [2]:
# Identifying pitched or flat roof of the buildings
# Calculating height for UC calculations and attic heights

def identify_roof_type(row):
    num_stories = row['num_stories']
    height_building = row['bldg_ht_m']

    # If num_stories does not have any value or is not numeric, assume it to be a shed of height 3m
    if pd.isnull(num_stories) or not isinstance(num_stories, (int, float)):
        num_stories = 1
        height_calculation = 3.0
        
    # If num_stories has value
    else:
        height_calculation = num_stories * 2.6
        
    # Checking roof types
    if height_building > height_calculation:
        roof_type = 'Pitched'
        # The half of the actual height has been considered in the dataset, therefore, multiplying it by 1.5 to get the actual height.
        attic_height = (max(0, height_building - height_calculation))* 1.5

        # Check if attic height is less than 2.0m, then recalculate
        # If the attic height is less than 2.0m, then the floor is included in the previous floor
        if roof_type == 'Pitched' and attic_height < 2.0:
            height_calculation = (num_stories - 1) * 2.6
            attic_height = max(0, height_building - height_calculation)
    else:
        roof_type = 'Flat'
        attic_height = 0

    return pd.Series({
        'roof_type': roof_type,
        'height_calculation': height_calculation,
        'attic_height': attic_height
    })


##### City Data

Following cells extract and clean Ithaca City Data.

In [3]:
# Fetching City Data and filtering required columns
buildings_df = pd.read_csv(r'D:\Circular Construction Lab\Datasets\City Data\AllData_R1.csv', usecols=[
    'LBuildingID', 'AddressCalc', 'AddressZIPCalc', 
    'ConstructionClassification', 'Foundation', 'Roof', 'SidingMaterial', 'YearBin_ESL', 
    'Lon', 'Lat', 
    'wallArchetypes_ESL', 'roofArchetypes_ESL',
    'HeightInStories', 'Perimeter_M_ESL', 'Area_SQM_ESL', 'BldgHeight_M_ESL'
])

# Renaming columns to make them more intuitive
new_column_names = {
    'LBuildingID': 'building_id',
    'AddressCalc': 'address',
    'AddressZIPCalc': 'zip_code',
    'ConstructionClassification': 'construction_classification',
    'Foundation': 'foundation_material',
    'Roof': 'roof_material',
    'SidingMaterial': 'siding_material',
    'YearBin_ESL': 'year_bin',
    'Lon': 'longitude',
    'Lat': 'latitude',
    'wallArchetypes_ESL': 'wall_archetypes',
    'roofArchetypes_ESL': 'roof_archetypes',
    'HeightInStories': 'num_stories',
    'Perimeter_M_ESL': 'perimeter_m',
    'Area_SQM_ESL': 'area_sqm',
    'BldgHeight_M_ESL': 'bldg_ht_m'
}

buildings_df = buildings_df.rename(columns=new_column_names)

# Adding rooftypes to the dataset
new_columns = buildings_df.apply(identify_roof_type, axis=1)
buildings_df = pd.concat([buildings_df, new_columns], axis=1)

# Calculating wall area (excluding the window area using the wall to window ratio of 20%)
buildings_df['wall_area_sqm'] = buildings_df['height_calculation'] * buildings_df['perimeter_m'] * 0.8

# Calculate roof area based on roof type
buildings_df['roof_area_sqm'] = buildings_df.apply(
    lambda row: row['perimeter_m'] * row['attic_height'] if row['roof_type'] == 'Pitched' else row['area_sqm'],
    axis=1)

# Drop rows where building IDs are NaNs
buildings_df = buildings_df.dropna(subset=['building_id'])

# Cleaning archetype columns
archetype_columns = ['wall_archetypes', 'roof_archetypes']
buildings_df[archetype_columns] = buildings_df[archetype_columns].apply(lambda x: x.str.replace(r"[\[\]']", "", regex=True))



This is the list of missing information.

Removing them to run the pipeline without error.

In [4]:
# Drop rows where wall_archetypes or roof_archetypes is 'None'
buildings_df = buildings_df[(buildings_df['wall_archetypes'] != 'None') & (buildings_df['roof_archetypes'] != 'None')]

# Drop rows where wall_archetypes is 'EW04B37'
buildings_df = buildings_df[(buildings_df['wall_archetypes'] != 'EW04B37')]

# Drop rows where wall_archetypes is 'EW04B35'
buildings_df = buildings_df[(buildings_df['wall_archetypes'] != 'EW04B35')]

# Drop rows where wall_archetypes is '-76.50684771154903 42.45064128743241'
buildings_df = buildings_df[(buildings_df['wall_archetypes'] != '-76.50684771154903 42.45064128743241')]

# Drop rows where roof_archetypes is '-76.50684771154903 42.45064128743241'
buildings_df = buildings_df[(buildings_df['roof_archetypes'] != '-76.50684771154903 42.45064128743241')]

# Drop rows with NaN values in wall_archetypes and roof_archetypes columns
buildings_df = buildings_df.dropna(subset=['wall_archetypes', 'roof_archetypes'])

# Drop rows where wall_archetypes or roof_archetypes is 'EW04B37'
buildings_df = buildings_df[(buildings_df['wall_archetypes'] != 'nan') & (buildings_df['roof_archetypes'] != 'nan')]

In [5]:
buildings_df

,building_id,address,zip_code,construction_classification,foundation_material,num_stories,roof_material,siding_material,longitude,latitude,...,wall_archetypes,roof_archetypes,perimeter_m,area_sqm,bldg_ht_m,roof_type,height_calculation,attic_height,wall_area_sqm,roof_area_sqm
3,11.48121,86 Mary Street,14850.0,NaN,NaN,NaN,NaN,NaN,-76.514848,42.433660,...,EW01B23,RF00A70,61.877504,117.517544,3.2,Pitched,0.0,3.2,0.000000,198.008011
4,11.37021,70 Mary Street (78.-3-6.5),14850.0,NaN,NaN,NaN,NaN,NaN,-76.514251,42.433665,...,EW01B23,RF00A70,58.687112,112.774976,2.9,Flat,3.0,0.0,140.849069,112.774976
5,11.16021,26 Jake Street (78.-3-6.82),14850.0,NaN,NaN,NaN,NaN,NaN,-76.512618,42.433581,...,EW01B23,RF00A70,66.047923,120.684007,3.0,Flat,3.0,0.0,158.515016,120.684007
8,11.57121,46 Ruben Street,14850.0,NaN,NaN,NaN,NaN,NaN,-76.513003,42.433535,...,EW01B23,RF00A70,62.032579,117.888801,3.7,Pitched,0.0,3.7,0.000000,229.520543
9,11.07121,17 Jake Street,14850.0,NaN,NaN,NaN,NaN,NaN,-76.512136,42.433545,...,EW01B23,RF00A70,58.135994,105.244447,3.2,Pitched,0.0,3.2,0.000000,186.035180
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7993,16841.00000,323 Plain Street N,14850.0,NaN,NaN,NaN,NaN,NaN,-76.503824,42.442243,...,EW01B23,RF00A87,87.115209,281.983154,3.7,Pitched,0.0,3.7,0.000000,322.326273
7994,11.70341,309 Plain Street N,14850.0,Type VB,Stone,2.0,Asphalt,Asbestos,-76.503836,42.441624,...,EW09E09,RF00A70,74.579736,224.340724,6.3,Pitched,2.6,3.7,155.125851,275.945024
7995,16844.00000,326 Buffalo Street W,14850.0,NaN,NaN,NaN,NaN,NaN,-76.503567,42.441423,...,EW01E09,RF00A70,72.394756,211.231851,NaN,Flat,3.0,0.0,173.747415,211.231851
7996,16820.00000,411-15 College Avenue,14850.0,NaN,NaN,NaN,NaN,NaN,-76.485006,42.442348,...,EW01B23,RF00A87,124.893548,785.185781,10.8,Pitched,3.0,11.7,299.744514,1461.254508


##### Organizing buildings_df into smaller subsets

In [6]:
# Create subsets of dataframes
metadata_df = buildings_df[['building_id', 'address', 'zip_code', 'longitude', 'latitude']]
construction_info_df = buildings_df[['construction_classification', 'foundation_material', 'siding_material',
                                      'roof_material', 'wall_archetypes', 'roof_archetypes']]
building_mensuration_df = buildings_df[['num_stories', 'perimeter_m', 'area_sqm', 'bldg_ht_m',
                                         'attic_height', 'wall_area_sqm', 'roof_area_sqm', 'height_calculation']]
building_style_df = buildings_df[['roof_type']]

# Store subsets in HDF5 file
with pd.HDFStore('building_data.h5', mode='w') as store:
    store['metadata'] = metadata_df
    store['construction_info'] = construction_info_df
    store['building_mensuration'] = building_mensuration_df
    store['building_style'] = building_style_df

##### Material Library

Urban Circularity uses the RhinoCircular Material Database.

Source: https://www.researchgate.net/publication/367207729_RhinoCircular_Material_Circularity_Database_V10

In [7]:
# Fetching the material library
material_library = pd.read_csv(r'D:\Circular Construction Lab\Datasets\UrbanCircularity_MatLib.csv')

# Dropping columns I don't require (Material ID)
material_library = material_library.drop(columns = ['Material ID'])

# Renaming columns for convenience
material_library = material_library.rename(columns= {'Density [kg/m3]': 'Density',
                                        'Prod. Recycled [%] Pc': 'Pc',
                                        'Prod. Renewable [%] Pn': 'Pn',
                                        'EoU Recycling [%] Sc': 'Sc',
                                        'EoU Composting [%] Sm': 'Sm',
                                        'Embodied Carbon [CO2/kg]': 'Ec',
                                        'Prod. Efficiency [%] Ep': 'Ep',
                                        'EoU Energy Rec. [%] Se': 'Se',
                                        'EoU Efficiency [%] Es': 'Es'        
                                        })

In [8]:
material_library.head()

,Name,Family,Density,Pc,Pn,Ep,Sc,Se,Sm,Es,Ec
0,Pine Wood,Timber,513,0.0,1.0,0.85,0.03,0.18,0.06,0.8,0.71
1,Douglas Fir,Timber,530,0.0,1.0,0.85,0.03,0.18,0.06,0.8,0.71
2,Oak,Timber,466,0.0,1.0,0.85,0.03,0.18,0.06,0.8,0.71
3,Spruce,Timber,525,0.0,1.0,0.85,0.03,0.18,0.06,0.8,0.71
4,Larch,Timber,570,0.0,1.0,0.85,0.03,0.18,0.06,0.8,0.71


##### Querying Construction Archetypes

Construction Archetypes are a standardized model or pattern representing typical construction details used in building projects. These archetypes encompass specific techniques, materials, and designs for building substructures such as walls, roofs, and foundations.

AllData.csv contains pre-assigned and these archetype are being queried from the local directory.

In [9]:
def find_archetype(i, type_archetype, metadata_df, construction_info_df):
    archetype_name = construction_info_df.iloc[i][type_archetype]

    # The archetype address will change with users
    archetypes_dir = r"D:\Circular Construction Lab\UrbanCircularity_Archetypes\UrbanCircularity_Archetypes"

    # Required columns from the dataset
    req_columns = ['Layer', 'Name', 'Layer Thickness', '%', 'Pui', 'Sui']

    # Creating an empty archetype DataFrame
    archetype_df = pd.DataFrame()

    # Defining a method to add a prefix to layer values and make them unique
    def add_prefix(archetype_name, value):
        return archetype_name + '_' + str(value)

    # Checking the archetype_name values for more than 1 value or 'None'
    if ',' in archetype_name:
        names = archetype_name.split(',')
        for name in names:
            clean_name = name.strip()
            archetype_add = os.path.join(archetypes_dir, clean_name + '.csv')
            archetype = pd.read_csv(archetype_add)
            archetype_df = archetype_df.append(archetype, ignore_index=True)
            archetype_df['Layer'] = archetype_df['Layer'].apply(lambda x: add_prefix(clean_name, x))
    else:
        clean_name = archetype_name
        archetype_add = os.path.join(archetypes_dir, clean_name + '.csv')
        archetype = pd.read_csv(archetype_add)
        archetype_df = archetype_df.append(archetype, ignore_index=True)
        archetype_df['Layer'] = archetype_df['Layer'].apply(lambda x: add_prefix(clean_name, x))

    archetype_df = archetype_df[req_columns]

    return archetype_df

In [10]:
find_archetype(6, 'wall_archetypes', pd.read_hdf('building_data.h5', key='metadata'), pd.read_hdf('building_data.h5', key='construction_info'))

,Layer,Name,Layer Thickness,%,Pui,Sui
0,EW01B23_1,Pine Wood,12.700,1.00000,0,0
1,EW01B23_2,Oriented Strand Board,12.000,1.00000,0,0
2,EW01B23_3,Mineral Wool Insulation,139.700,0.90625,0,0
3,EW01B23_3,Pine Wood,139.700,0.09375,0,0
4,EW01B23_4,High Density Polyethylene (HDPE),6.000,1.00000,0,0
5,EW01B23_5,Gypsum Board,9.525,1.00000,0,0


##### 3. Calculating material mass values for each archetype per building

Various mass values will be used in further calculations.

In [11]:
def calc_massValues(x, type_archetype, metadata_df, construction_info_df, material_library, building_mensuration):
    # Find the archetype
    archetype_df = find_archetype(x, type_archetype, metadata_df, construction_info_df)
    
    # Fetch building-specific data
    building_id = metadata_df.iloc[x]['building_id']
    building_dims = building_mensuration.iloc[x]
    
    # Storing values in a variable layer_values
    layer_values = []
    for _, row in archetype_df.iterrows():
        layer_material_name = row['Name']
        layer_thickness = row['Layer Thickness'] / 1000
        
        # Filter data from material library based on layer material names
        mat_info = material_library.loc[material_library['Name'] == layer_material_name]

        # Calculate volume
        if (type_archetype == 'wall_archetypes'):
            volume = layer_thickness * row['%'] * building_dims['wall_area_sqm']
        elif (type_archetype == 'roof_archetypes'):
            volume = layer_thickness * row['%'] * building_dims['roof_area_sqm']       
            
        if not mat_info.empty:
            material_family = mat_info['Family'].iloc[0]
            mat_density = mat_info['Density'].iloc[0]
            material_name = mat_info['Name'].iloc[0]
            
            pct_reused = row['Pui']
            pct_recycled = 1 - pct_reused

            layer_mass = volume * mat_density
            reused_mass = layer_mass * pct_reused
            recycled_mass = (layer_mass - reused_mass) * mat_info['Pc'].iloc[0]
            renewable_mass = (layer_mass - reused_mass) * mat_info['Pn'].iloc[0]
            reusable_mass = layer_mass * row['Sui']
            compostable_mass = (layer_mass - reusable_mass) * mat_info['Sm'].iloc[0]
            recyclable_mass = (layer_mass - reusable_mass) * mat_info['Sc'].iloc[0]
            biodegradable_mass = (layer_mass - reused_mass) * mat_info['Se'].iloc[0]
            ep = mat_info['Ep'].iloc[0]
            landfill_production = layer_mass * (((1 - ep) * mat_info['Pc'].iloc[0]) / ep)
            es = mat_info['Es'].iloc[0]
            landfill_end = recyclable_mass * (1 - es)
            embodied_carbon = layer_mass * mat_info['Ec'].iloc[0]

            layer_values.append([
                f"{building_id}_{type_archetype}_{row['Layer']}", 
                material_family,
                material_name,
                volume,
                layer_mass,
                reused_mass,
                recycled_mass,
                renewable_mass,
                reusable_mass,
                compostable_mass,
                recyclable_mass,
                biodegradable_mass,
                landfill_production,
                landfill_end,
                embodied_carbon
            ])

    columns = [
        'bldg_archlayer', 'material_family', 'material_name', 'volume', 'layer_mass', 'reused_mass', 'recycled_mass',
        'renewable_mass', 'reusable_mass', 'compostable_mass', 'recyclable_mass', 'biodegradable_mass',
        'landfill_production', 'landfill_end', 'embodied_carbon'
    ]
    return pd.DataFrame(layer_values, columns=columns)


In [12]:
calc_massValues(0, 'wall_archetypes', pd.read_hdf('building_data.h5', key='metadata'), pd.read_hdf('building_data.h5', key='construction_info'), 
                material_library, pd.read_hdf('building_data.h5', key='building_mensuration'))

,bldg_archlayer,material_family,material_name,volume,layer_mass,reused_mass,recycled_mass,renewable_mass,reusable_mass,compostable_mass,recyclable_mass,biodegradable_mass,landfill_production,landfill_end,embodied_carbon
0,11.48121_wall_archetypes_EW01B23_1,Timber,Pine Wood,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,11.48121_wall_archetypes_EW01B23_2,Timber,Oriented Strand Board,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,11.48121_wall_archetypes_EW01B23_3,Mineral,Mineral Wool Insulation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,11.48121_wall_archetypes_EW01B23_3,Timber,Pine Wood,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,11.48121_wall_archetypes_EW01B23_4,Plastic,High Density Polyethylene (HDPE),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,11.48121_wall_archetypes_EW01B23_5,Mineral,Gypsum Board,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


##### 4. Calculating Building Material Stock

##### Declaring materials groups, specific materials, and archetypes.

In [13]:
# Define material families and archetype names
material_families = ['Timber', 'Glass', 'Plastic', 'Metal', 'Mineral', 'Organic']
archetype_names = ['wall_archetypes', 'roof_archetypes']
    
# Define material type lists
material_types = {
        'Timber': ['Pine Wood', 'Douglas Fir', 'Oak', 'Spruce', 'Larch', 'Oriented Strand Board', 'Medium Density Fiberboard'],
        'Mineral': ['Concrete', 'Light Weight Concrete', 'Recycling Concrete', 'Concrete Hollow Block', 'Asphalt Shingles',
                     'Cement Screed', 'Brick', 'Adobe', 'Rammed Earth', 'Natural Stone', 'Mineral Wool Insulation', 'Gypsum Board', 'Ceramics', 'Asbestos'],
        'Organic': ['Wood Fiber Insulating Board', 'Cork', 'Cellulose Insulation'],
        'Glass': ['Foam Glass', 'Glass'],
        'Metal': ['Reinforcement Bars', 'Stainless Steel', 'Steel', 'Copper', 'Aluminium'],
        'Plastic': ['High Density Polyethylene (HDPE)', 'Low Density Polyethylene (LDPE)', 'Extruded Polystyrene Foam (XPS)',
                     'Polyurethane Insulation (PUR)', 'Polyvinyl Chloride (PVC)', 'Carpet']
    }

##### Calculate total building masses

In [14]:
def calc_bldg_masses(metadata_df, archetype_names, construction_info_df, material_library, building_mensuration):
    # Initialize lists to store total masses
    total_mass_bldg_list = []
    total_mass_wall_list = []
    total_mass_roof_list = []
    
    # Iterate through the DataFrame rows
    for i in range(len(metadata_df)):
        total_mass_bldg = 0
        total_mass_wall = 0
        total_mass_roof = 0
        
        # Extract mass values for the current row and archetype
        for archetype_name in archetype_names:
            massValues = calc_massValues(i, archetype_name, metadata_df, construction_info_df, material_library, building_mensuration)

            # Sum up the layer masses
            layer_masses = massValues['layer_mass'].sum()
            
            total_mass_bldg += layer_masses

            if archetype_name == 'wall_archetypes':
                total_mass_wall += layer_masses
            elif archetype_name == 'roof_archetypes':
                total_mass_roof += layer_masses
        
        # Append total masses to lists
        total_mass_bldg_list.append(total_mass_bldg)
        total_mass_wall_list.append(total_mass_wall)
        total_mass_roof_list.append(total_mass_roof)
    
    # Create a new dataframe for total masses
    total_mass_df = pd.DataFrame({
        'total_mass_bldg': total_mass_bldg_list,
        'total_mass_wall': total_mass_wall_list,
        'total_mass_roof': total_mass_roof_list
    })
    
    # Store the new dataframe in the HDF5 file
    with pd.HDFStore('building_data.h5', mode='a') as store:
        store['total_mass'] = total_mass_df


In [15]:
calc_bldg_masses( pd.read_hdf('building_data.h5', key='metadata'), archetype_names,  
                 pd.read_hdf('building_data.h5', key='construction_info'), material_library, pd.read_hdf('building_data.h5', key='building_mensuration'))

In [16]:
pd.read_hdf('building_data.h5', key='total_mass')

,total_mass_bldg,total_mass_wall,total_mass_roof
0,5601.262382,0.000000,5601.262382
1,9372.444798,6182.259575,3190.185223
2,10371.583737,6957.667387,3413.916350
3,6492.690743,0.000000,6492.690743
4,5262.574206,0.000000,5262.574206
...,...,...,...
5311,19661.671501,0.000000,19661.671501
5312,12176.797016,4370.847798,7805.949217
5313,13419.247947,7443.908815,5975.339132
5314,102292.102088,13156.625088,89135.477000


##### Calculate total building embodied carbon footprint

In [17]:
def calc_bldg_embodied_carbon(metadata_df, archetype_names, construction_info_df, material_library, building_mensuration):
    # Initialize lists to store total embodied carbon
    total_ec_bldg_list = []
    total_ec_wall_list = []
    total_ec_roof_list = []
    
    # Iterate through the DataFrame rows
    for i in range(len(metadata_df)):
        total_ec_bldg = 0
        total_ec_wall = 0
        total_ec_roof = 0
        
        # Extract mass values for the current row and archetype
        for archetype_name in archetype_names:
            massValues = calc_massValues(i, archetype_name, metadata_df, construction_info_df, material_library, building_mensuration)

            # Sum up the embodied carbon values
            total_ec = massValues['embodied_carbon'].sum()
            
            total_ec_bldg += total_ec

            if archetype_name == 'wall_archetypes':
                total_ec_wall += total_ec
            elif archetype_name == 'roof_archetypes':
                total_ec_roof += total_ec
        
        # Append total embodied carbon to lists
        total_ec_bldg_list.append(total_ec_bldg)
        total_ec_wall_list.append(total_ec_wall)
        total_ec_roof_list.append(total_ec_roof)
    
    # Create a new dataframe for total embodied carbon
    total_ec_df = pd.DataFrame({
        'total_ec_bldg': total_ec_bldg_list,
        'total_ec_wall': total_ec_wall_list,
        'total_ec_roof': total_ec_roof_list
    })
    
    # Store the new dataframe in the HDF5 file
    with pd.HDFStore('building_data.h5', mode='a') as store:
        store['total_embodied_carbon'] = total_ec_df

In [18]:
calc_bldg_embodied_carbon( pd.read_hdf('building_data.h5', key='metadata'), archetype_names,  
                          pd.read_hdf('building_data.h5', key='construction_info'), 
                          material_library, pd.read_hdf('building_data.h5', key='building_mensuration'))

In [19]:
pd.read_hdf('building_data.h5', key='total_embodied_carbon')

,total_ec_bldg,total_ec_wall,total_ec_roof
0,3976.896291,0.000000,3976.896291
1,8927.328571,6662.297063,2265.031508
2,9921.794023,7497.913414,2423.880608
3,4609.810428,0.000000,4609.810428
4,3736.427686,0.000000,3736.427686
...,...,...,...
5311,19412.391858,0.000000,19412.391858
5312,9731.269224,4189.045279,5542.223944
5313,14495.961814,10253.471030,4242.490783
5314,102183.581924,14178.205172,88005.376751


##### Calculate total material masses

In [20]:
def initialize_material_mass_dataframes(material_families, archetype_names, material_types, metadata_df):
    # Create an empty dictionary to store dataframes for each material family
    material_family_dfs = {}
    
    # Get the number of rows from the metadata DataFrame
    num_rows = len(metadata_df)
    
    # Iterate over material families
    for material_family in material_families:
        # Create a list of column names for the material family
        df_columns = [f'total_mass_{material_family}'] + \
                     [f'total_mass_{material_family}_{archetype_name}' for archetype_name in archetype_names] + \
                     [f'total_mass_{material_family}_{material_name}' for material_name in material_types[material_family]] + \
                     [f'total_mass_{material_family}_{material_name}_{archetype_name}' for material_name in material_types[material_family] for archetype_name in archetype_names]
        
        # Create a DataFrame with the specified columns and initialize all values to zero
        df = pd.DataFrame(0, index=range(num_rows), columns=df_columns)
        
        # Add the DataFrame to the dictionary with the material family as the key
        material_family_dfs[material_family] = df
    
    return material_family_dfs

In [ ]:
def update_material_mass_dataframes(material_family_dfs, material_families, archetype_names, material_types,
                                    metadata_df, construction_info_df, building_mensuration, material_library):
    # Iterate through the DataFrame rows
    for i in range(len(metadata_df)):
        # Extract archetype dataframe
        for archetype_name in archetype_names:
            massValues = calc_massValues(i, archetype_name, metadata_df, construction_info_df, material_library, building_mensuration)

            # Iterate through materials in the archetype
            for j in range(len(massValues)):
                material_family = massValues.iloc[j]['material_family']
                material_name = massValues.iloc[j]['material_name']
                layer_mass = massValues.iloc[j]['layer_mass']
                
                if material_family in material_families:
                    # Access the DataFrame for the material family once
                    material_df = material_family_dfs[material_family]
                    
                    # Update the corresponding columns in the material family dataframe
                    material_df.loc[i, f'total_mass_{material_family}'] += layer_mass
                    material_df.loc[i, f'total_mass_{material_family}_{archetype_name}'] += layer_mass

                    if material_name in material_types[material_family]:
                        material_df.loc[i, f'total_mass_{material_family}_{material_name}'] += layer_mass
                        material_df.loc[i, f'total_mass_{material_family}_{material_name}_{archetype_name}'] += layer_mass
    
    return material_family_dfs

# Initialize material mass dataframes
metadata_df = pd.read_hdf('building_data.h5', key='metadata')
construction_info_df = pd.read_hdf('building_data.h5', key='construction_info')
building_mensuration = pd.read_hdf('building_data.h5', key='building_mensuration')

material_family_dfs = initialize_material_mass_dataframes(material_families, archetype_names, material_types, metadata_df)

# Update material mass dataframes with data
material_family_dfs = update_material_mass_dataframes(material_family_dfs, material_families, archetype_names, material_types, 
                                metadata_df, construction_info_df, building_mensuration, material_library)

# Store each dataframe in the HDF5 file
with pd.HDFStore('building_data.h5', mode='a') as store:
    for material_family, df in material_family_dfs.items():
        store[f'total_{material_family}_mass'] = df

In [22]:
with pd.HDFStore('building_data.h5', mode='r') as store:
    print(store.keys())


['/building_mensuration', '/building_style', '/construction_info', '/metadata', '/total_Glass_mass', '/total_Metal_mass', '/total_Mineral_mass', '/total_Organic_mass', '/total_Plastic_mass', '/total_Timber_mass', '/total_embodied_carbon', '/total_mass']


In [23]:
pd.read_hdf('building_data.h5', key='total_Timber_mass')

,total_mass_Timber,total_mass_Timber_wall_archetypes,total_mass_Timber_roof_archetypes,total_mass_Timber_Pine Wood,total_mass_Timber_Pine Wood_wall_archetypes,total_mass_Timber_Pine Wood_roof_archetypes,total_mass_Timber_Douglas Fir,total_mass_Timber_Douglas Fir_wall_archetypes,total_mass_Timber_Douglas Fir_roof_archetypes,total_mass_Timber_Oak,...,total_mass_Timber_Spruce_roof_archetypes,total_mass_Timber_Larch,total_mass_Timber_Larch_wall_archetypes,total_mass_Timber_Larch_roof_archetypes,total_mass_Timber_Oriented Strand Board,total_mass_Timber_Oriented Strand Board_wall_archetypes,total_mass_Timber_Oriented Strand Board_roof_archetypes,total_mass_Timber_Medium Density Fiberboard,total_mass_Timber_Medium Density Fiberboard_wall_archetypes,total_mass_Timber_Medium Density Fiberboard_roof_archetypes
0,5601.262382,0.000000,5601.262382,3789.498360,0.000000,3789.498360,1811.764022,0,1811.764022,0,...,0,0,0,0,0.000000,0.000000,0.000000,0,0,0
1,6102.070265,2911.885042,3190.185223,4022.267446,1863.967969,2158.299477,1031.885746,0,1031.885746,0,...,0,0,0,0,1047.917073,1047.917073,0.000000,0,0,0
2,6691.023587,3277.107237,3413.916350,4407.418865,2097.755520,2309.663345,1104.253005,0,1104.253005,0,...,0,0,0,0,1179.351717,1179.351717,0.000000,0,0,0
3,6492.690743,0.000000,6492.690743,4392.588535,0.000000,4392.588535,2100.102208,0,2100.102208,0,...,0,0,0,0,0.000000,0.000000,0.000000,0,0,0
4,5262.574206,0.000000,5262.574206,3560.361033,0.000000,3560.361033,1702.213173,0,1702.213173,0,...,0,0,0,0,0.000000,0.000000,0.000000,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5311,8044.633734,0.000000,8044.633734,3248.418787,0.000000,3248.418787,0.000000,0,0.000000,0,...,0,0,0,0,4796.214946,0.000000,4796.214946,0,0,0
5312,10616.848547,2810.899330,7805.949217,8091.964514,2810.899330,5281.065184,2524.884033,0,2524.884033,0,...,0,0,0,0,0.000000,0.000000,0.000000,0,0,0
5313,8486.923711,2511.584579,5975.339132,6554.162179,2511.584579,4042.577600,1932.761531,0,1932.761531,0,...,0,0,0,0,0.000000,0.000000,0.000000,0,0,0
5314,42666.913433,6196.857207,36470.056226,18693.347165,3966.758019,14726.589146,0.000000,0,0.000000,0,...,0,0,0,0,23973.566268,2230.099188,21743.467080,0,0,0


##### Calculate total building masses

In [24]:
def initialize_material_ec_dataframes(material_families, archetype_names, material_types, metadata_df):
    # Create an empty dictionary to store dataframes for each material family
    material_family_dfs = {}
    
    # Get the number of rows from the metadata DataFrame
    num_rows = len(metadata_df)
    
    # Iterate over material families
    for material_family in material_families:
        # Create a list of column names for the material family
        df_columns = [f'total_ec_{material_family}'] + \
                     [f'total_ec_{material_family}_{archetype_name}' for archetype_name in archetype_names] + \
                     [f'total_ec_{material_family}_{material_name}' for material_name in material_types[material_family]] + \
                     [f'total_ec_{material_family}_{material_name}_{archetype_name}' for material_name in material_types[material_family] for archetype_name in archetype_names]
        
        # Create a DataFrame with the specified columns and initialize all values to zero
        df = pd.DataFrame(0, index=range(num_rows), columns=df_columns)
        
        # Add the DataFrame to the dictionary with the material family as the key
        material_family_dfs[material_family] = df
    
    return material_family_dfs

In [ ]:
def update_material_ec_dataframes(material_family_dfs, material_families, archetype_names, material_types,
                                  metadata_df, construction_info_df, building_mensuration, material_library):
    # Iterate through the DataFrame rows
    for i in range(len(metadata_df)):
        # Extract archetype dataframe
        for archetype_name in archetype_names:
            massValues = calc_massValues(i, archetype_name, metadata_df, construction_info_df, material_library, building_mensuration)

            # Iterate through materials in the archetype
            for j in range(len(massValues)):
                material_family = massValues.iloc[j]['material_family']
                material_name = massValues.iloc[j]['material_name']
                embodied_carbon = massValues.iloc[j]['embodied_carbon']
                
                if material_family in material_families:
                    # Access the DataFrame for the material family once
                    material_df = material_family_dfs[material_family]
                    
                    # Update the corresponding columns in the material family dataframe
                    material_df.loc[i, f'total_ec_{material_family}'] += embodied_carbon
                    material_df.loc[i, f'total_ec_{material_family}_{archetype_name}'] += embodied_carbon

                    if material_name in material_types[material_family]:
                        material_df.loc[i, f'total_ec_{material_family}_{material_name}'] += embodied_carbon
                        material_df.loc[i, f'total_ec_{material_family}_{material_name}_{archetype_name}'] += embodied_carbon
    
    return material_family_dfs

# Initialize material embodied carbon dataframes
metadata_df = pd.read_hdf('building_data.h5', key='metadata')
construction_info_df = pd.read_hdf('building_data.h5', key='construction_info')
building_mensuration = pd.read_hdf('building_data.h5', key='building_mensuration')

material_family_dfs = initialize_material_ec_dataframes(material_families, archetype_names, material_types, metadata_df)

# Update material embodied carbon dataframes with data
material_family_dfs = update_material_ec_dataframes(material_family_dfs, material_families, archetype_names, material_types, 
                                metadata_df, construction_info_df, building_mensuration, material_library)

# Store each dataframe in the HDF5 file
with pd.HDFStore('building_data.h5', mode='a') as store:
    for material_family, df in material_family_dfs.items():
        store[f'total_{material_family}_ec'] = df


In [26]:
with pd.HDFStore('building_data.h5', mode='r') as store:
    print(store.keys())

['/building_mensuration', '/building_style', '/construction_info', '/metadata', '/total_Glass_ec', '/total_Glass_mass', '/total_Metal_ec', '/total_Metal_mass', '/total_Mineral_ec', '/total_Mineral_mass', '/total_Organic_ec', '/total_Organic_mass', '/total_Plastic_ec', '/total_Plastic_mass', '/total_Timber_ec', '/total_Timber_mass', '/total_embodied_carbon', '/total_mass']


In [27]:
pd.read_hdf('building_data.h5', key='total_Timber_ec')

,total_ec_Timber,total_ec_Timber_wall_archetypes,total_ec_Timber_roof_archetypes,total_ec_Timber_Pine Wood,total_ec_Timber_Pine Wood_wall_archetypes,total_ec_Timber_Pine Wood_roof_archetypes,total_ec_Timber_Douglas Fir,total_ec_Timber_Douglas Fir_wall_archetypes,total_ec_Timber_Douglas Fir_roof_archetypes,total_ec_Timber_Oak,...,total_ec_Timber_Spruce_roof_archetypes,total_ec_Timber_Larch,total_ec_Timber_Larch_wall_archetypes,total_ec_Timber_Larch_roof_archetypes,total_ec_Timber_Oriented Strand Board,total_ec_Timber_Oriented Strand Board_wall_archetypes,total_ec_Timber_Oriented Strand Board_roof_archetypes,total_ec_Timber_Medium Density Fiberboard,total_ec_Timber_Medium Density Fiberboard_wall_archetypes,total_ec_Timber_Medium Density Fiberboard_roof_archetypes
0,3976.896291,0.000000,3976.896291,2690.543835,0.000000,2690.543835,1286.352456,0,1286.352456,0,...,0,0,0,0,0.000000,0.000000,0.000000,0,0,0
1,4594.449157,2329.417648,2265.031508,2855.809887,1323.417258,1532.392629,732.638880,0,732.638880,0,...,0,0,0,0,1006.000390,1006.000390,0.000000,0,0,0
2,5045.464676,2621.584067,2423.880608,3129.267394,1489.406419,1639.860975,784.019634,0,784.019634,0,...,0,0,0,0,1132.177648,1132.177648,0.000000,0,0,0
3,4609.810428,0.000000,4609.810428,3118.737860,0.000000,3118.737860,1491.072568,0,1491.072568,0,...,0,0,0,0,0.000000,0.000000,0.000000,0,0,0
4,3736.427686,0.000000,3736.427686,2527.856334,0.000000,2527.856334,1208.571353,0,1208.571353,0,...,0,0,0,0,0.000000,0.000000,0.000000,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5311,6910.743687,0.000000,6910.743687,2306.377339,0.000000,2306.377339,0.000000,0,0.000000,0,...,0,0,0,0,4604.366348,0.000000,4604.366348,0,0,0
5312,7537.962469,1995.738524,5542.223944,5745.294805,1995.738524,3749.556281,1792.667663,0,1792.667663,0,...,0,0,0,0,0.000000,0.000000,0.000000,0,0,0
5313,6025.715835,1783.225051,4242.490783,4653.455147,1783.225051,2870.230096,1372.260687,0,1372.260687,0,...,0,0,0,0,0.000000,0.000000,0.000000,0,0,0
5314,36286.900104,4957.293414,31329.606691,13272.276487,2816.398194,10455.878293,0.000000,0,0.000000,0,...,0,0,0,0,23014.623617,2140.895220,20873.728397,0,0,0


##### Calculate Cp (mass of circular materials in manufacturing of item/sub-assembly/assembly) [kg]

In [21]:
def calc_bldg_cps(metadata_df, archetype_names, construction_info_df, material_library, building_mensuration):
    # Initialize lists to store total cps
    total_cp_bldg_list = []
    total_cp_wall_list = []
    total_cp_roof_list = []
    
    # Iterate through the DataFrame rows
    for i in range(len(metadata_df)):
        total_cp_bldg = 0
        total_cp_wall = 0
        total_cp_roof = 0
        
        # Extract archetype dataframe
        for archetype_name in archetype_names:
            massValues = calc_massValues(i, archetype_name, metadata_df, construction_info_df, material_library, building_mensuration)
            
             # Iterate through layers of materials
            for j in range(len(massValues)):
                reused_mass = massValues.iloc[j]['reused_mass']
                recycled_mass = massValues.iloc[j]['recycled_mass']
                renewable_mass = massValues.iloc[j]['renewable_mass']
                
                # Calculates cp values
                cp = reused_mass + recycled_mass + renewable_mass
                
                # Update building cp
                total_cp_bldg += cp

                # Update total sums for wall and roof
                if archetype_name == 'wall_archetypes':
                    total_cp_wall += cp
                elif archetype_name == 'roof_archetypes':
                    total_cp_roof += cp
        
        # Append total cp to lists
        total_cp_bldg_list.append(total_cp_bldg)
        total_cp_wall_list.append(total_cp_wall)
        total_cp_roof_list.append(total_cp_roof)
    
    # Create a new dataframe for total cp
    total_cp_df = pd.DataFrame({
        'total_cp_bldg': total_cp_bldg_list,
        'total_cp_wall': total_cp_wall_list,
        'total_cp_roof': total_cp_roof_list
    })
    
    # Store the new dataframe in the HDF5 file
    with pd.HDFStore('building_data.h5', mode='a') as store:
        store['total_cp'] = total_cp_df

In [ ]:
def calc_bldg_cps(metadata_df, archetype_names, construction_info_df, material_library, building_mensuration):
    # Initialize lists to store total cps
    total_cp_bldg_list = []
    total_cp_wall_list = []
    total_cp_roof_list = []
    
    # Iterate through the DataFrame rows
    for i in range(len(metadata_df)):
        total_cp_bldg = 0
        total_cp_wall = 0
        total_cp_roof = 0
        
        # Extract archetype dataframe
        for archetype_name in archetype_names:
            mass_values = calc_massValues(i, archetype_name, metadata_df, construction_info_df, material_library, building_mensuration)
            
            # Calculate cp values for all layers
            cp_values = mass_values[['reused_mass', 'recycled_mass', 'renewable_mass']].sum(axis=1)
            total_cp = cp_values.sum()
            
            # Update building cp
            total_cp_bldg += total_cp
            
            # Update total sums for wall and roof
            if archetype_name == 'wall_archetypes':
                total_cp_wall += total_cp
            elif archetype_name == 'roof_archetypes':
                total_cp_roof += total_cp
        
        # Append total cp to lists
        total_cp_bldg_list.append(total_cp_bldg)
        total_cp_wall_list.append(total_cp_wall)
        total_cp_roof_list.append(total_cp_roof)
    
    # Create a new dataframe for total cp
    total_cp_df = pd.DataFrame({
        'total_cp_bldg': total_cp_bldg_list,
        'total_cp_wall': total_cp_wall_list,
        'total_cp_roof': total_cp_roof_list
    })
    
    # Store the new dataframe in the HDF5 file
    with pd.HDFStore('building_data.h5', mode='a') as store:
        store['total_cp'] = total_cp_df

In [22]:
calc_bldg_cps( pd.read_hdf('building_data.h5', key='metadata'), archetype_names,  
                 pd.read_hdf('building_data.h5', key='construction_info'), material_library, pd.read_hdf('building_data.h5', key='building_mensuration'))

##### Calculate Cp/materials (mass of circular materials in manufacturing of item/sub-assembly/assembly) [kg]

In [23]:
def initialize_material_cp_dataframes(material_families, archetype_names, material_types, metadata_df):
    # Create an empty dictionary to store dataframes for each material family
    material_family_dfs = {}
    
    # Get the number of rows from the metadata DataFrame
    num_rows = len(metadata_df)
    
    # Iterate over material families
    for material_family in material_families:
        # Create a list of column names for the material family
        df_columns = [f'total_cp_{material_family}']
        
        # Add columns for each archetype
        df_columns += [f'total_cp_{material_family}_{archetype_name}' for archetype_name in archetype_names]
        
        # Add columns for each material name within the family and each archetype
        for material_name in material_types[material_family]:
            df_columns.append(f'total_cp_{material_family}_{material_name}')
            df_columns += [f'total_cp_{material_family}_{material_name}_{archetype_name}' for archetype_name in archetype_names]
        
        # Create a DataFrame with the specified columns and initialize all values to zero
        df = pd.DataFrame(0, index=range(num_rows), columns=df_columns)
        
        # Add the DataFrame to the dictionary with the material family as the key
        material_family_dfs[material_family] = df
    
    return material_family_dfs

In [ ]:
def update_material_cp_dataframes(material_family_dfs, material_families, archetype_names, material_types,
                                  metadata_df, construction_info_df, building_mensuration, material_library):
    # Iterate through the DataFrame rows
    for i in range(len(metadata_df)):
        # Extract archetype dataframe
        for archetype_name in archetype_names:
            mass_values = calc_massValues(i, archetype_name, metadata_df, construction_info_df, material_library, building_mensuration)
            
            # Calculate cp values for all layers
            mass_values['cp'] = mass_values[['reused_mass', 'recycled_mass', 'renewable_mass']].sum(axis=1)
            
            # Group by material family and material name
            grouped = mass_values.groupby(['material_family', 'material_name'])['cp'].sum().reset_index()
            
            # Update the material family dataframes
            for _, row in grouped.iterrows():
                material_family = row['material_family']
                material_name = row['material_name']
                cp = row['cp']
                
                if material_family in material_families:
                    material_df = material_family_dfs[material_family]
                    
                    material_df.at[i, f'total_cp_{material_family}'] += cp
                    material_df.at[i, f'total_cp_{material_family}_{archetype_name}'] += cp

                    if material_name in material_types[material_family]:
                        material_df.at[i, f'total_cp_{material_family}_{material_name}'] += cp
                        material_df.at[i, f'total_cp_{material_family}_{material_name}_{archetype_name}'] += cp
    
    return material_family_dfs

# Initialize material cp dataframes
material_family_dfs = initialize_material_cp_dataframes(material_families, archetype_names, material_types, pd.read_hdf('building_data.h5', key='metadata'))

# Update material cp dataframes with data
material_family_dfs = update_material_cp_dataframes(material_family_dfs, material_families, archetype_names, material_types, 
                                pd.read_hdf('building_data.h5', key='metadata'), pd.read_hdf('building_data.h5', key='construction_info'), 
                                pd.read_hdf('building_data.h5', key='building_mensuration'), 
                                material_library)

# Store each dataframe in the HDF5 file
with pd.HDFStore('building_data.h5', mode='a') as store:
    for material_family, df in material_family_dfs.items():
        store[f'total_{material_family}_cp'] = df


##### Calculate Cs (mass of circular materials in end-of-service of item/sub-assembly/assembly) [kg]

In [ ]:
def calc_bldg_css(metadata_df, archetype_names, construction_info_df, material_library, building_mensuration):
    # Initialize lists to store total cs
    total_cs_bldg_list = []
    total_cs_wall_list = []
    total_cs_roof_list = []
    
    # Iterate through the DataFrame rows
    for i in range(len(metadata_df)):
        total_cs_bldg = 0
        total_cs_wall = 0
        total_cs_roof = 0
        
        # Iterate through each archetype
        for archetype_name in archetype_names:
            mass_values = calc_massValues(i, archetype_name, metadata_df, construction_info_df, material_library, building_mensuration)
            
            # Calculate cs values for all layers
            mass_values['cs'] = mass_values[['reusable_mass', 'compostable_mass', 'recyclable_mass']].sum(axis=1)
            
            # Sum up the cs values
            total_cs_archetype = mass_values['cs'].sum()
            
            # Update building cs
            total_cs_bldg += total_cs_archetype

            # Update total sums for wall and roof
            if archetype_name == 'wall_archetypes':
                total_cs_wall += total_cs_archetype
            elif archetype_name == 'roof_archetypes':
                total_cs_roof += total_cs_archetype
        
        # Append total cs to lists
        total_cs_bldg_list.append(total_cs_bldg)
        total_cs_wall_list.append(total_cs_wall)
        total_cs_roof_list.append(total_cs_roof)
    
    # Create a new dataframe for total cs
    total_cs_df = pd.DataFrame({
        'total_cs_bldg': total_cs_bldg_list,
        'total_cs_wall': total_cs_wall_list,
        'total_cs_roof': total_cs_roof_list
    })
    
    # Store the new dataframe in the HDF5 file
    with pd.HDFStore('building_data.h5', mode='a') as store:
        store['total_cs'] = total_cs_df

In [33]:
calc_bldg_css( pd.read_hdf('building_data.h5', key='metadata'), archetype_names,  
                 pd.read_hdf('building_data.h5', key='construction_info'), material_library, pd.read_hdf('building_data.h5', key='building_mensuration'))

##### Calculate Cs/material (mass of circular materials in end-of-service of item/sub-assembly/assembly) [kg]

In [34]:
def initialize_material_cp_dataframes(material_families, archetype_names, material_types, metadata_df):
    # Create an empty dictionary to store dataframes for each material family
    material_family_dfs = {}
    
    # Get the number of rows from the metadata DataFrame
    num_rows = len(metadata_df)
    
    # Iterate over material families
    for material_family in material_families:
        # Create a list of column names for the material family
        df_columns = [f'total_cs_{material_family}']
        
        # Add columns for each archetype
        df_columns += [f'total_cs_{material_family}_{archetype_name}' for archetype_name in archetype_names]
        
        # Add columns for each material name within the family and each archetype
        for material_name in material_types[material_family]:
            df_columns.append(f'total_cs_{material_family}_{material_name}')
            df_columns += [f'total_cs_{material_family}_{material_name}_{archetype_name}' for archetype_name in archetype_names]
        
        # Create a DataFrame with the specified columns and initialize all values to zero
        df = pd.DataFrame(0, index=range(num_rows), columns=df_columns)
        
        # Add the DataFrame to the dictionary with the material family as the key
        material_family_dfs[material_family] = df
    
    return material_family_dfs

In [ ]:
def update_material_cs_dataframes(material_family_dfs, material_families, archetype_names, material_types,
                                  metadata_df, construction_info_df, building_mensuration, material_library):
    # Iterate through the DataFrame rows
    for i in range(len(metadata_df)):
        # Iterate through each archetype name
        for archetype_name in archetype_names:
            # Calculate mass values for the current index and archetype
            mass_values = calc_massValues(i, archetype_name, metadata_df, construction_info_df, material_library, building_mensuration)
            
            # Calculate cs values for all layers
            mass_values['cs'] = mass_values[['reusable_mass', 'compostable_mass', 'recyclable_mass']].sum(axis=1)
            
            # Group by material family and material name
            grouped = mass_values.groupby(['material_family', 'material_name'])['cs'].sum().reset_index()
            
            # Update the material family dataframes
            for _, row in grouped.iterrows():
                material_family = row['material_family']
                material_name = row['material_name']
                cs = row['cs']
                
                if material_family in material_families:
                    material_df = material_family_dfs[material_family]
                    
                    material_df.at[i, f'total_cs_{material_family}'] += cs
                    material_df.at[i, f'total_cs_{material_family}_{archetype_name}'] += cs

                    if material_name in material_types[material_family]:
                        material_df.at[i, f'total_cs_{material_family}_{material_name}'] += cs
                        material_df.at[i, f'total_cs_{material_family}_{material_name}_{archetype_name}'] += cs
                    
                    # Update the dataframe in the dictionary
                    material_family_dfs[material_family] = material_df
    
    return material_family_dfs

metadata_df = pd.read_hdf('building_data.h5', key='metadata')
construction_info_df = pd.read_hdf('building_data.h5', key='construction_info')
building_mensuration = pd.read_hdf('building_data.h5', key='building_mensuration')
    
# Initialize material cs dataframes
material_family_dfs = initialize_material_cs_dataframes(material_families, archetype_names, material_types, metadata_df)

# Update material cs dataframes with data
material_family_dfs = update_material_cs_dataframes(material_family_dfs, material_families, archetype_names, material_types, 
                                                        metadata_df, construction_info_df, building_mensuration, material_library)

# Store each dataframe in the HDF5 file
with pd.HDFStore('building_data.h5', mode='a') as store:
    for material_family, df in material_family_dfs.items():
        store[f'total_{material_family}_cs'] = df

 ##### Calculate  Wsc (mass of waste from the recycling process of item/sub-assembly/assembly at end-of-service) [kg]

In [36]:
def calc_bldg_wsc(metadata_df, archetype_names, construction_info_df, material_library, building_mensuration):
    # Initialize lists to store total wsc
    total_wsc_bldg_list = []
    total_wsc_wall_list = []
    total_wsc_roof_list = []
    
    # Iterate through the DataFrame rows
    for i in range(len(metadata_df)):
        total_wsc_bldg = 0
        total_wsc_wall = 0
        total_wsc_roof = 0
        
        # Extract archetype dataframe
        for archetype_name in archetype_names:
            mass_values = calc_massValues(i, archetype_name, metadata_df, construction_info_df, material_library, building_mensuration)
            
            # Calculate total wsc for the current archetype
            wsc_archetype = mass_values['landfill_end'].sum()
            
            # Update building mass
            total_wsc_bldg += wsc_archetype

            # Update total sums for wall and roof
            if archetype_name == 'wall_archetypes':
                total_wsc_wall += wsc_archetype
            elif archetype_name == 'roof_archetypes':
                total_wsc_roof += wsc_archetype
        
        # Append total wsc to lists
        total_wsc_bldg_list.append(total_wsc_bldg)
        total_wsc_wall_list.append(total_wsc_wall)
        total_wsc_roof_list.append(total_wsc_roof)
    
    # Create a new dataframe for total wsc
    total_wsc_df = pd.DataFrame({
        'total_wsc_bldg': total_wsc_bldg_list,
        'total_wsc_wall': total_wsc_wall_list,
        'total_wsc_roof': total_wsc_roof_list
    })
    
    # Store the new dataframe in the HDF5 file
    with pd.HDFStore('building_data.h5', mode='a') as store:
        store['total_wsc'] = total_wsc_df

In [37]:
calc_bldg_wsc( pd.read_hdf('building_data.h5', key='metadata'), archetype_names,  
                 pd.read_hdf('building_data.h5', key='construction_info'), material_library, pd.read_hdf('building_data.h5', key='building_mensuration'))

 ##### Calculate  Wsc/material (mass of waste from the recycling process of item/sub-assembly/assembly at end-of-service) [kg]

In [38]:
def initialize_material_wsc_dataframes(material_families, archetype_names, material_types, metadata_df):
    # Create an empty dictionary to store dataframes for each material family
    material_family_dfs = {}
    
    # Get the number of rows from the metadata DataFrame
    num_rows = len(metadata_df)
    
    # Iterate over material families
    for material_family in material_families:
        # Create a list of column names for the material family
        df_columns = [f'total_wsc_{material_family}']
        
        # Add columns for each archetype
        for archetype_name in archetype_names:
            df_columns.append(f'total_wsc_{material_family}_{archetype_name}')
        
        # Add columns for each material name within the family and each archetype
        for material_name in material_types[material_family]:
            df_columns.extend([
                f'total_wsc_{material_family}_{material_name}', 
            ])
            for archetype_name in archetype_names:
                df_columns.append(f'total_wsc_{material_family}_{material_name}_{archetype_name}')
        
        # Create a DataFrame with the specified columns and initialize all values to zero
        df = pd.DataFrame(0, index=range(num_rows), columns=df_columns)
        
        # Add the DataFrame to the dictionary with the material family as the key
        material_family_dfs[material_family] = df
    
    return material_family_dfs

In [40]:
def update_material_wsc_dataframes(material_family_dfs, material_families, archetype_names, material_types,
                                    metadata_df, construction_info_df, building_mensuration, material_library):
    # Iterate through the DataFrame rows
    for i in range(len(metadata_df)):
        # Extract archetype dataframe
        for archetype_name in archetype_names:
            massValues = calc_massValues(i, archetype_name, metadata_df, construction_info_df, material_library, building_mensuration)

            # Iterate through materials in the archetype
            for j in range(len(massValues)):
                material_family = massValues.iloc[j]['material_family']
                material_name = massValues.iloc[j]['material_name']
                landfill_end = massValues.iloc[j]['landfill_end']
                
                # Calculates wsc values
                wsc = landfill_end
                
                if material_family in material_families:
                    # Ensure that the DataFrame has a row for the current index
                    material_df = material_family_dfs[material_family]
                        
                    # Update the corresponding columns in the material family dataframe
                    material_df.at[i, f'total_wsc_{material_family}'] += wsc
                    material_df.at[i, f'total_wsc_{material_family}_{archetype_name}'] += wsc

                    if material_name in material_types[material_family]:
                        material_df.at[i, f'total_wsc_{material_family}_{material_name}'] += wsc
                        material_df.at[i, f'total_wsc_{material_family}_{material_name}_{archetype_name}'] += wsc
                        
                    # Update the dataframe in the dictionary
                    material_family_dfs[material_family] = material_df
    
    return material_family_dfs

# Initialize material mass dataframes
material_family_dfs = initialize_material_wsc_dataframes(material_families, archetype_names, material_types, pd.read_hdf('building_data.h5', key='metadata'))

# Update material mass dataframes with data
material_family_dfs = update_material_wsc_dataframes(material_family_dfs, material_families, archetype_names, material_types, 
                                pd.read_hdf('building_data.h5', key='metadata'), pd.read_hdf('building_data.h5', key='construction_info'), 
                                pd.read_hdf('building_data.h5', key='building_mensuration'), 
                                material_library)

# Store each dataframe in the HDF5 file
with pd.HDFStore('building_data.h5', mode='a') as store:
    for material_family, df in material_family_dfs.items():
        store[f'total_{material_family}_wsc'] = df

 ##### Calculate  Wpc (mass of waste from the recycling process in production of item/sub-assembly/assembly) [kg]

In [44]:
def calc_bldg_wpc(metadata_df, archetype_names, construction_info_df, material_library, building_mensuration):
    # Initialize lists to store total wpcs
    total_wpc_bldg_list = []
    total_wpc_wall_list = []
    total_wpc_roof_list = []
    
    # Iterate through the DataFrame rows
    for i in range(len(metadata_df)):
        total_wpc_bldg = 0
        total_wpc_wall = 0
        total_wpc_roof = 0
        
        # Extract archetype dataframe
        for archetype_name in archetype_names:
            massValues = calc_massValues(i, archetype_name, metadata_df, construction_info_df, material_library, building_mensuration)
            
             # Iterate through layers of materials
            for j in range(len(massValues)):
                landfill_production = massValues['landfill_production']
                
                # Calculates wpc values
                wpc = landfill_production
                
                # Update building mass
                total_wpc_bldg += wpc

                # Update total sums for wall and roof
                if archetype_name == 'wall_archetypes':
                    total_wpc_wall += wpc
                elif archetype_name == 'roof_archetypes':
                    total_wpc_roof += wpc
        
        # Append total wpcs to lists
        total_wpc_bldg_list.append(total_wpc_bldg)
        total_wpc_wall_list.append(total_wpc_wall)
        total_wpc_roof_list.append(total_wpc_roof)
    
    # Create a new dataframe for total wpcs
    total_wpc_df = pd.DataFrame({
        'total_wpc_bldg': total_wpc_bldg_list,
        'total_wpc_wall': total_wpc_wall_list,
        'total_wpc_roof': total_wpc_roof_list
    })
    
    # Store the new dataframe in the HDF5 file
    with pd.HDFStore('building_data.h5', mode='a') as store:
        store['total_wpc'] = total_wpc_df

 ##### Calculate  Wpc/material (mass of waste from the recycling process of item/sub-assembly/assembly at end-of-service) [kg]

In [49]:
def initialize_material_wpc_dataframes(material_families, archetype_names, material_types, metadata_df):
    # Create an empty dictionary to store dataframes for each material family
    material_family_dfs = {}
    
    # Get the number of rows from the metadata DataFrame
    num_rows = len(metadata_df)
    
    # Iterate over material families
    for material_family in material_families:
        # Create a list of column names for the material family
        df_columns = [f'total_wpc_{material_family}']
        
        # Add columns for each archetype
        for archetype_name in archetype_names:
            df_columns.append(f'total_wpc_{material_family}_{archetype_name}')
        
        # Add columns for each material name within the family and each archetype
        for material_name in material_types[material_family]:
            df_columns.extend([
                f'total_wpc_{material_family}_{material_name}', 
            ])
            for archetype_name in archetype_names:
                df_columns.append(f'total_wpc_{material_family}_{material_name}_{archetype_name}')
        
        # Create a DataFrame with the specified columns and initialize all values to zero (float)
        df = pd.DataFrame(0.0, index=range(num_rows), columns=df_columns)
        
        # Add the DataFrame to the dictionary with the material family as the key
        material_family_dfs[material_family] = df
    
    return material_family_dfs


In [50]:
def update_material_wpc_dataframes(material_family_dfs, material_families, archetype_names, material_types,
                                    metadata_df, construction_info_df, building_mensuration, material_library):
    # Iterate through the DataFrame rows
    for i in range(len(metadata_df)):
        # Extract archetype dataframe
        for archetype_name in archetype_names:
            massValues = calc_massValues(i, archetype_name, metadata_df, construction_info_df, material_library, building_mensuration)

            # Iterate through materials in the archetype
            for j in range(len(massValues)):
                material_family = massValues.iloc[j]['material_family']
                material_name = massValues.iloc[j]['material_name']
                landfill_production = massValues.iloc[j]['landfill_production']
                
                # Calculates wsc values
                wpc = float(landfill_production)  # Ensure wpc is a float
                
                if material_family in material_families:
                    # Ensure that the DataFrame has a row for the current index
                    material_df = material_family_dfs[material_family]
                        
                    # Update the corresponding columns in the material family dataframe
                    material_df.at[i, f'total_wpc_{material_family}'] += wpc
                    material_df.at[i, f'total_wpc_{material_family}_{archetype_name}'] += wpc

                    if material_name in material_types[material_family]:
                        material_df.at[i, f'total_wpc_{material_family}_{material_name}'] += wpc
                        material_df.at[i, f'total_wpc_{material_family}_{material_name}_{archetype_name}'] += wpc
    
    return material_family_dfs


# Load the data
metadata_df = pd.read_hdf('building_data.h5', key='metadata')
construction_info_df = pd.read_hdf('building_data.h5', key='construction_info')
building_mensuration = pd.read_hdf('building_data.h5', key='building_mensuration')

# Initialize material mass dataframes
material_family_dfs = initialize_material_wpc_dataframes(material_families, archetype_names, material_types, metadata_df)

# Update material mass dataframes with data
material_family_dfs = update_material_wpc_dataframes(material_family_dfs, material_families, archetype_names, material_types, 
                                metadata_df, construction_info_df, building_mensuration, 
                                material_library)

# Store each dataframe in the HDF5 file
with pd.HDFStore('building_data.h5', mode='a') as store:
    for material_family, df in material_family_dfs.items():
        store[f'total_{material_family}_wpc'] = df


In [ ]:
with pd.HDFStore('building_data.h5', mode='r') as store:
    print(store.keys())

##### 5. Calculating Circularity Indexes

 ##### Calculate CIp (Circularity Indicator for the production of item/sub-assembly-assembly) [%]

In [ ]:
def calc_CIPs(total_mass_df, total_mass_timber_df, total_mass_mineral_df, total_mass_organic_df,):
    # Create an empty DataFrame
    cip_df = pd.DataFrame()
    
    # Indices required for calculating and storing CIPs
    custom_indices_cip = [
        ('total_bldg_cip', 'total_cp_building', 'total_mass_building'),
        ('total_Timber_cip', 'total_cp_Timber', 'total_mass_Timber'),
        ('total_Mineral_cip', 'total_cp_Mineral', 'total_mass_Mineral'),
        ('total_Organic_cip', 'total_cp_Oranic', 'total_mass_Organic'),
        ('total_Glass_cip', 'total_cp_Glass', 'total_mass_Glass'),
        ('total_Metal_cip', 'total_cp_Metal', 'total_mass_Metal'),
        ('total_Plastic_cip', 'total_cp_Plastic', 'total_mass_Plastic')
        # Add more custom indices as needed in the format ('result_col', 'numerator_col', 'denominator_col')
    ]

        
    # Calculating CIPs and storing it in the dataset
    for result_col, numerator_col, denominator_col in custom_indices_cip:
        cip_df[result_col] = cip_df.apply(lambda row: (row[numerator_col] / row[denominator_col]) * 100 if not np.isnan(row[numerator_col]) and row[numerator_col] != 0 else np.nan, 
                                          axis=1)
    
    # Store the new dataframe in the HDF5 file
    with pd.HDFStore('building_data.h5', mode='a') as store:
        store['total_cip'] = cip_df

 ##### Calculate CIs (Circularity Indicator for end-of-service of item/sub-assembly/assembly) [%]

In [ ]:
def calc_CISs():
    # Create an empty DataFrame
    cis_df = pd.DataFrame()
    
    # Indices required for calculating and storing CIPs
    custom_indices_cis = [
        ('total_bldg_cis', 'total_cs_building', 'total_wsc_building', 'total_mass_building'),
        ('total_Timber_cis', 'total_cs_Timber', 'total_wsc_Timber', 'total_mass_Timber'),
        ('total_Mineral_cis', 'total_cs_Mineral', 'total_wsc_Mineral', 'total_mass_Mineral'),
        ('total_Organic_cis', 'total_cs_Oranic', 'total_wsc_Organic', 'total_mass_Organic'),
        ('total_Glass_cis', 'total_cs_Glass', 'total_wsc_Glass', 'total_mass_Glass'),
        ('total_Metal_cis', 'total_cs_Metal', 'total_wsc_Metal', 'total_mass_Metal'),
        ('total_Plastic_cis', 'total_cs_Plastic', 'total_wsc_Plastic', 'total_mass_Plastic')
        # Add more custom indices as needed in the format ('result_col', 'numerator_col', 'denominator_col')
    ]
        
    # Calculating CISs and storing it in the dataset    
    for result_col, numerator_col_1, numerator_col_2, denominator_col in custom_indices_cis:
        cis_df[result_col] = cis_df.apply(
            lambda row: ((row[numerator_col_1] - row[numerator_col_2])/ row[denominator_col]) * 100 if not np.isnan((row[numerator_col_1] - row[numerator_col_2])) 
            and (row[numerator_col_1] - row[numerator_col_2]) != 0 else np.nan, axis=1)
    
    # Store the new dataframe in the HDF5 file
    with pd.HDFStore('building_data.h5', mode='a') as store:
        store['total_cis'] = cis_df

 ##### Calculate CItotal (Circularity Indicator for total building and for total materials/building) [%]

In [ ]:
def calc_CItotals():
    # Create an empty DataFrame
    ci_totals_df = pd.DataFrame()
    
    # Indices required for calculating and storing W (mass of waste from item/sub-assembly/assembly)
    custom_indices_w = [
        ('w', 'total_mass_building', 'total_cs_building', 'total_wpc_building', 'total_wsc_building'),
        ('w_Timber', 'total_mass_Timber', 'total_cs_Timber', 'total_wpc_Timber', 'total_wsc_Timber'),
        ('w_Mineral', 'total_mass_Mineral', 'total_cs_Mineral', 'total_wpc_Mineral', 'total_wsc_Mineral'),
        ('w_Organic', 'total_mass_Organic', 'total_cs_Organic', 'total_wpc_Organic', 'total_wsc_Organic'),
        ('w_Glass', 'total_mass_Glass', 'total_cs_Glass', 'total_wpc_Glass', 'total_wsc_Glass'),
        ('w_Metal', 'total_mass_Metal', 'total_cs_Metal', 'total_wpc_Metal', 'total_wsc_Metal'),
        ('w_Plastic', 'total_mass_Plastic', 'total_cs_Plastic', 'total_wpc_Plastic', 'total_wsc_Plastic'),
        # Add more custom indices as needed in the format ('result_col', 'numerator_col', 'denominator_col')
    ]
    
    for result_col, mass_parameter, cs_parameter, wpc_parameter, wsc_parameter in custom_indices_w:
        ci_totals_df[result_col] = ci_totals_df.apply(lambda row: (row[mass_parameter] - row[cs_parameter]) + ((row[wpc_parameter] + row[wsc_parameter])/2), axis=1)
        
        
    custom_indices_lfi = [('lfi', 'total_mass_building', 'total_cp_building', 'w', 'total_wpc_building', 'total_wsc_building', 'total_ci'),
                         ('lfi_timber', 'total_mass_Timber', 'total_cp_Timber', 'w_Timber', 'total_wpc_Timber', 'total_wsc_Timber', 'total_ci_Timber'),
                         ('lfi_mineral', 'total_mass_Mineral', 'total_cp_Mineral', 'w_Mineral', 'total_wpc_Mineral', 'total_wsc_Mineral', 'total_ci_Mineral'),
                         ('lfi_organic', 'total_mass_Organic', 'total_cp_Organic', 'w_Organic', 'total_wpc_Organic', 'total_wsc_Organic', 'total_ci_Organic'),
                         ('lfi_glass', 'total_mass_Glass', 'total_cp_Glass', 'w_Glass', 'total_wpc_Glass', 'total_wsc_Glass', 'total_ci_Glass'),
                         ('lfi_metal', 'total_mass_Metal', 'total_cp_Metal', 'w_Metal', 'total_wpc_Metal', 'total_wsc_Metal', 'total_ci_Metal'),
                         ('lfi_plastic', 'total_mass_Plastic', 'total_cp_Plastic', 'w_Plastic', 'total_wpc_Plastic', 'total_wsc_Plastic', 'total_ci_Plastic'),
                         # Add indices as needed in this format ('result_col', 'mass_parameter', 'cp_parameter', 'w_parameter' 'wpc_parameter', 'wsc_parameter', 'ci_parameter')
                         ]
    for result_col, mass_parameter, cp_parameter, w_parameter, wpc_parameter, wsc_parameter, ci_parameter in custom_indices_lfi:
        ci_totals_df[result_col] = ci_totals_df.apply(
            lambda row: ((row[mass_parameter] - row[cp_parameter]) + row[w_parameter]) / (2 * row[mass_parameter] + ((row[wpc_parameter] - row[wsc_parameter])/2))
            if not np.isnan((row[mass_parameter] - row[cp_parameter]) + row[w_parameter]) and ((row[mass_parameter] - row[cp_parameter]) + row[w_parameter]) != 0 else np.nan,
            axis=1)
        ci_totals_df[ci_parameter] = ci_totals_df.apply(lambda row: (1 - row[result_col]), axis=1)
    
    # Store the new dataframe in the HDF5 file
    with pd.HDFStore('building_data.h5', mode='a') as store:
        store['total_ci_totals'] = ci_totals_df

 ##### Calculate CI for pathways  (Circularity Indicator for total extra masses) [%]

 ##### Calculate total mass values for different mass pathways

In [ ]:
def calculate_mass_totals(metadata_df, archetype_names, construction_info_df, material_library, building_mensuration):
    # Initialize lists to store totals
    total_reused_mass_list = []
    total_recycled_mass_list = []
    total_renewable_mass_list = []
    total_reusable_mass_list = []
    total_recyclable_mass_list = []
    total_compostable_mass_list = []
    total_reused_mass_wall_list = []
    total_recycled_mass_wall_list = []
    total_renewable_mass_wall_list = []
    total_reusable_mass_wall_list = []
    total_recyclable_mass_wall_list = []
    total_compostable_mass_wall_list = []
    total_reused_mass_roof_list = []
    total_recycled_mass_roof_list = []
    total_renewable_mass_roof_list = []
    total_reusable_mass_roof_list = []
    total_recyclable_mass_roof_list = []
    total_compostable_mass_roof_list = []

    for i in range(len(metadata_df)):
        # Initialize totals for this row
        total_reused_mass = 0
        total_recycled_mass = 0
        total_renewable_mass = 0
        total_reusable_mass = 0
        total_recyclable_mass = 0
        total_compostable_mass = 0
        total_reused_mass_wall = 0
        total_recycled_mass_wall = 0
        total_renewable_mass_wall = 0
        total_reusable_mass_wall = 0
        total_recyclable_mass_wall = 0
        total_compostable_mass_wall = 0
        total_reused_mass_roof = 0
        total_recycled_mass_roof = 0
        total_renewable_mass_roof = 0
        total_reusable_mass_roof = 0
        total_recyclable_mass_roof = 0
        total_compostable_mass_roof = 0

        # Extract mass values for the current row and archetype
        for archetype_name in archetype_names:
            massValues = calc_massValues(i, archetype_name, metadata_df, construction_info_df, material_library, building_mensuration)

            # Sum up the masses
            reused_mass = massValues['reused_mass'].sum()
            recycled_mass = massValues['recycled_mass'].sum()
            renewable_mass = massValues['renewable_mass'].sum()
            reusable_mass = massValues['reusable_mass'].sum()
            recyclable_mass = massValues['recyclable_mass'].sum()
            compostable_mass = massValues['compostable_mass'].sum()

            total_reused_mass += reused_mass
            total_recycled_mass += recycled_mass
            total_renewable_mass += renewable_mass
            total_reusable_mass += reusable_mass
            total_recyclable_mass += recyclable_mass
            total_compostable_mass += compostable_mass

            if archetype_name == 'wall_archetypes':
                total_reused_mass_wall += reused_mass
                total_recycled_mass_wall += recycled_mass
                total_renewable_mass_wall += renewable_mass
                total_reusable_mass_wall += reusable_mass
                total_recyclable_mass_wall += recyclable_mass
                total_compostable_mass_wall += compostable_mass
            elif archetype_name == 'roof_archetypes':
                total_reused_mass_roof += reused_mass
                total_recycled_mass_roof += recycled_mass
                total_renewable_mass_roof += renewable_mass
                total_reusable_mass_roof += reusable_mass
                total_recyclable_mass_roof += recyclable_mass
                total_compostable_mass_roof += compostable_mass

        # Append totals to lists
        total_reused_mass_list.append(total_reused_mass)
        total_recycled_mass_list.append(total_recycled_mass)
        total_renewable_mass_list.append(total_renewable_mass)
        total_reusable_mass_list.append(total_reusable_mass)
        total_recyclable_mass_list.append(total_recyclable_mass)
        total_compostable_mass_list.append(total_compostable_mass)
        total_reused_mass_wall_list.append(total_reused_mass_wall)
        total_recycled_mass_wall_list.append(total_recycled_mass_wall)
        total_renewable_mass_wall_list.append(total_renewable_mass_wall)
        total_reusable_mass_wall_list.append(total_reusable_mass_wall)
        total_recyclable_mass_wall_list.append(total_recyclable_mass_wall)
        total_compostable_mass_wall_list.append(total_compostable_mass_wall)
        total_reused_mass_roof_list.append(total_reused_mass_roof)
        total_recycled_mass_roof_list.append(total_recycled_mass_roof)
        total_renewable_mass_roof_list.append(total_renewable_mass_roof)
        total_reusable_mass_roof_list.append(total_reusable_mass_roof)
        total_recyclable_mass_roof_list.append(total_recyclable_mass_roof)
        total_compostable_mass_roof_list.append(total_compostable_mass_roof)

    # Create a new dataframe for total masses
    total_ex_mass_df = pd.DataFrame({
        'total_reused_mass': total_reused_mass_list,
        'total_recycled_mass': total_recycled_mass_list,
        'total_renewable_mass': total_renewable_mass_list,
        'total_reusable_mass': total_reusable_mass_list,
        'total_recyclable_mass': total_recyclable_mass_list,
        'total_compostable_mass': total_compostable_mass_list,
        'total_reused_mass_wall': total_reused_mass_wall_list,
        'total_recycled_mass_wall': total_recycled_mass_wall_list,
        'total_renewable_mass_wall': total_renewable_mass_wall_list,
        'total_reusable_mass_wall': total_reusable_mass_wall_list,
        'total_recyclable_mass_wall': total_recyclable_mass_wall_list,
        'total_compostable_mass_wall': total_compostable_mass_wall_list,
        'total_reused_mass_roof': total_reused_mass_roof_list,
        'total_recycled_mass_roof': total_recycled_mass_roof_list,
        'total_renewable_mass_roof': total_renewable_mass_roof_list,
        'total_reusable_mass_roof': total_reusable_mass_roof_list,
        'total_recyclable_mass_roof': total_recyclable_mass_roof_list,
        'total_compostable_mass_roof': total_compostable_mass_roof_list,
    })

    # Store the new dataframe in the HDF5 file
    with pd.HDFStore('building_data.h5', mode='a') as store:
        store['total_ex_masses'] = total_ex_mass_df

 ##### Calculate total CI values for different mass pathways

In [ ]:
# Change names for dataframes
def calc_total_ex_mass_CI(total_ex_mass_df, metadata_df, total_mass_df):
    
    # Create an empty DataFrame
    ci_totals_ex_mass_df = pd.DataFrame()
    
    # Calculating building total virgin and landfill values
    ci_totals_ex_mass_df['total_virgin_building'] = total_ex_mass_df['total_mass_building'] - total_ex_mass_df['total_cp_building']
    ci_totals_ex_mass_df['total_landfill_building'] = total_ex_mass_df['total_mass_building'] - total_ex_mass_df['total_cs_building']
    
    # Indices required for calculating CIs
    custom_indices_ci1 = [('reused_ci', 'total_reused_mass_wall', 'total_reused_mass_roof', 'total_mass_building'),
                         ('recycled_ci', 'total_recycled_mass_wall', 'total_recycled_mass_roof', 'total_mass_building'),
                         ('renewable_ci', 'total_renewable_mass_wall', 'total_renewable_mass_roof', 'total_mass_building'),
                         ('reusable_ci', 'total_reusable_mass_wall', 'total_reusable_mass_roof', 'total_mass_building'),
                         ('recyclable_ci', 'total_recyclable_mass_wall', 'total_recyclable_mass_roof', 'total_mass_building'),
                         ('compostable_ci', 'total_compostable_mass_wall', 'total_compostable_mass_roof', 'total_mass_building')]
    
    custom_indices_ci2 = [('virgin_ci', 'total_virgin_building', 'total_mass_building'),
                         ('landfill_ci', 'total_landfill_building', 'total_mass_building')]
    
    # Calculating CIs
    for result_col, wall_parameter, roof_parameter, mass_parameter in custom_indices_ci1:
         building_df[result_col] = building_df.apply(
            lambda row: ((row[wall_parameter] + row[roof_parameter]) / row[mass_parameter])
            if not np.isnan(row[wall_parameter] + row[roof_parameter]) and (row[wall_parameter] + row[roof_parameter]) != 0 else np.nan,
            axis=1
        )
            
    for result_col, extra_parameter, mass_parameter in custom_indices_ci2:
         building_df[result_col] = building_df.apply(
            lambda row: ((row[extra_parameter]) / row[mass_parameter])
            if not np.isnan(row[extra_parameter]) and (row[extra_parameter]) != 0 else np.nan,
            axis=1
        )
     # Store the new dataframe in the HDF5 file
    with pd.HDFStore('building_data.h5', mode='a') as store:
        store['ci_totals_ex_mass'] = ci_totals_ex_mass_df